In [1]:
from Utils.loadset import getDataSet
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import tensorflow_probability as tfp
import os
physical_devices = tf.config.list_physical_devices('GPU') 
print("Num GPUs:", len(physical_devices)) 

from trainer import Trainer
try:
    from Utils.connection_cfg import *
except Exception as e:
    PSWD = None
    USRN = None
    
from Utils.Data import dataWrapper, provideData
from Utils.transform import cutOut


tfd = tfp.distributions
def NLL(y_true, y_hat):
    return -y_hat.log_prob(y_true)

Num GPUs: 1
Num GPUs Available: 1


Using TensorFlow backend.


In [2]:
dimension = (512,512)

channels = 5

slices = [256,256+512,256,256+512]
slices_label = [128,128+256,128,128+256]
cutOutFrame = cutOut(slices)
cutOutFrame_label = cutOut(slices_label)
years = [2017]
PRETRAINING_TRANSFORMATIONS = [cutOutFrame]
TRANSFORMATION = [cutOutFrame_label]
batch_size = 25

train, test = provideData(dimension=dimension,
                          batch_size=batch_size,
                          channels = channels,
                          timeToPred = 5,
                          transform=TRANSFORMATION,
                          onlyUseYears=[2017],
                          preTransformation=PRETRAINING_TRANSFORMATIONS)

Found Year :  2017 => won't download this year again... please check for consistency
Finished Loading Dataset
 
[DEBUG] Data
[DEBUG] time to predict:  5
SORTTING OUUUT 61997
[DEBUG] Data
[DEBUG] time to predict:  5
SORTTING OUUUT 22305


In [3]:
import numpy as np
def negative_Binomial_Unet_BigBrainTime(input_shape,
                           activation_hidden="relu",
                           activation_output="relu"):
    
    inputs = Input(shape=input_shape) 

    conv00 = Conv2D(5, kernel_size=(5, 5), dilation_rate=3,padding="same")(inputs)
    conv00 = Activation(activation_hidden)(conv00)
    conv00 = Conv2D(10, kernel_size=(5, 5), strides=4,padding="same")(conv00)
    conv00 = Activation(activation_hidden)(conv00)
    
    
    conv01 = Conv2D(5, kernel_size=(5, 5), strides=(2,2),padding="same")(conv00)       
    conv01 = Activation(activation_hidden)(conv01)
    
    
    dilated_01 = Conv2D(5, kernel_size=(5, 5),padding="same",dilation_rate=2)(conv00)
    dilated_01 = Activation(activation_hidden)(dilated_01)
    dilated_01 = Conv2D(5, kernel_size=(3, 3), strides=(2,2),padding="same")(dilated_01)       
    dilated_01 = Activation(activation_hidden)(dilated_01)
    
    concatenate_01 = concatenate([conv01, dilated_01], axis=3)
    
    conv02 = Conv2D(10, kernel_size=(5, 5), strides=(2,2),padding="same")(concatenate_01)       
    conv02 = Activation(activation_hidden)(conv02)
    
    dilated_02 = Conv2D(10, kernel_size=(5, 5),padding="same",dilation_rate=2)(concatenate_01)
    dilated_02 = Activation(activation_hidden)(dilated_02)
    dilated_02 = Conv2D(10, kernel_size=(3, 3), strides=(2,2),padding="same")(dilated_02)       
    dilated_02 = Activation(activation_hidden)(dilated_02)
    
    concatenate_02 = concatenate([conv02, dilated_02], axis=3)
    
    conv03 = Conv2D(10, kernel_size=(5, 5), strides=(2,2),padding="same")(concatenate_02)       
    conv03 = Activation(activation_hidden)(conv03)
    
    dilated_03 = Conv2D(10, kernel_size=(5, 5),padding="same",dilation_rate=2)(concatenate_02)
    dilated_03 = Activation(activation_hidden)(dilated_03)
    dilated_03 = Conv2D(10, kernel_size=(3, 3), strides=(2,2),padding="same")(dilated_03)       
    dilated_03 = Activation(activation_hidden)(dilated_03)
    
    concatenate_03 = concatenate([conv03, dilated_03], axis=3)
    
    transpose_03 = Conv2DTranspose(10, kernel_size=(5, 5),dilation_rate=(3,3))(concatenate_03)       
    transpose_03 = Activation(activation_hidden)(transpose_03)
    transpose_03 = Conv2DTranspose(10, kernel_size=(3, 3),dilation_rate=(2,2))(transpose_03)       
    transpose_03 = Activation(activation_hidden)(transpose_03)
         
    concatenate_04 = concatenate([concatenate_02, transpose_03], axis=3)

    transpose_04 = Conv2DTranspose(10, kernel_size=(5, 5),dilation_rate=(3,3))(concatenate_04)       
    transpose_04 = Activation(activation_hidden)(transpose_04)
    transpose_04 = Conv2DTranspose(10, kernel_size=(5, 5),dilation_rate=(3,3))(transpose_04)       
    transpose_04 = Activation(activation_hidden)(transpose_04)
    transpose_04 = Conv2DTranspose(10, kernel_size=(3, 3),dilation_rate=(2,2))(transpose_04)       
    transpose_04 = Activation(activation_hidden)(transpose_04)
    transpose_04 = Conv2DTranspose(1, kernel_size=(3, 3),dilation_rate=(2,2))(transpose_04)       
    transpose_04 = Activation(activation_hidden)(transpose_04)
    
    concatenate_05 = concatenate([concatenate_01, transpose_04], axis=3)
    #concatenate_05 = Conv2D(1, kernel_size=(1, 1),padding="same")(concatenate_05)
    output = concatenate_05
    
    max_count = output[:,:,:,:1]
    prob      = output[:,:,:,1:2]
    max_count = Flatten()(max_count)
    prob      = Flatten()(prob)
    prob      = Dense(8*8)(prob)
    #prob      = Dropout(0.1)(prob)
    
    max_count      = Dense(10*10)(max_count)
    #max_count      = Dropout(0.1)(max_count)
    
    prob      = Dense(256*256,activation="sigmoid")(prob)
    max_count = Dense(256*256,activation="sigmoid")(max_count)
    prob      = tf.keras.layers.Reshape((256,256,1))(prob)
    max_count = tf.keras.layers.Reshape((256,256,1))(max_count)
    #max_count = Activation(activation_hidden)(max_count)
    input_dist= tf.concat([tf.math.softplus(max_count),prob],axis=-1)
    
    #print(max_count)
    
    output_dist = tfp.layers.DistributionLambda(name="DistributionLayer",
        make_distribution_fn=lambda t: tfd.Independent(tfp.distributions.NegativeBinomial(
            total_count=t[...,0:1] , probs=t[..., 1:2] ),
                                                      reinterpreted_batch_ndims=-1 ))
    output = output_dist(input_dist)
    #output = output_dist(prob)
    model = Model(inputs=inputs, outputs=output)

    
    return model


def Indentpendent_Poisson_Unet_BigBrainTime(input_shape,
                           activation_hidden="relu",
                           activation_output="relu"):
    
    inputs = Input(shape=input_shape) 

    conv00 = Conv2D(5, kernel_size=(5, 5), dilation_rate=3,padding="same")(inputs)
    conv00 = Activation(activation_hidden)(conv00)
    conv00 = Conv2D(10, kernel_size=(5, 5), strides=4,padding="same")(conv00)
    conv00 = Activation(activation_hidden)(conv00)
    
    
    conv01 = Conv2D(5, kernel_size=(5, 5), strides=(2,2),padding="same")(conv00)       
    conv01 = Activation(activation_hidden)(conv01)
    
    
    dilated_01 = Conv2D(5, kernel_size=(5, 5),padding="same",dilation_rate=2)(conv00)
    dilated_01 = Activation(activation_hidden)(dilated_01)
    dilated_01 = Conv2D(5, kernel_size=(3, 3), strides=(2,2),padding="same")(dilated_01)       
    dilated_01 = Activation(activation_hidden)(dilated_01)
    
    concatenate_01 = concatenate([conv01, dilated_01], axis=3)
    
    conv02 = Conv2D(10, kernel_size=(5, 5), strides=(2,2),padding="same")(concatenate_01)       
    conv02 = Activation(activation_hidden)(conv02)
    
    dilated_02 = Conv2D(10, kernel_size=(5, 5),padding="same",dilation_rate=2)(concatenate_01)
    dilated_02 = Activation(activation_hidden)(dilated_02)
    dilated_02 = Conv2D(10, kernel_size=(3, 3), strides=(2,2),padding="same")(dilated_02)       
    dilated_02 = Activation(activation_hidden)(dilated_02)
    
    concatenate_02 = concatenate([conv02, dilated_02], axis=3)
    
    conv03 = Conv2D(10, kernel_size=(5, 5), strides=(2,2),padding="same")(concatenate_02)       
    conv03 = Activation(activation_hidden)(conv03)
    
    dilated_03 = Conv2D(10, kernel_size=(5, 5),padding="same",dilation_rate=2)(concatenate_02)
    dilated_03 = Activation(activation_hidden)(dilated_03)
    dilated_03 = Conv2D(10, kernel_size=(3, 3), strides=(2,2),padding="same")(dilated_03)       
    dilated_03 = Activation(activation_hidden)(dilated_03)
    
    concatenate_03 = concatenate([conv03, dilated_03], axis=3)
    
    transpose_03 = Conv2DTranspose(10, kernel_size=(5, 5),dilation_rate=(3,3))(concatenate_03)       
    transpose_03 = Activation(activation_hidden)(transpose_03)
    transpose_03 = Conv2DTranspose(10, kernel_size=(3, 3),dilation_rate=(2,2))(transpose_03)       
    transpose_03 = Activation(activation_hidden)(transpose_03)
         
    concatenate_04 = concatenate([concatenate_02, transpose_03], axis=3)

    transpose_04 = Conv2DTranspose(10, kernel_size=(5, 5),dilation_rate=(3,3))(concatenate_04)       
    transpose_04 = Activation(activation_hidden)(transpose_04)
    transpose_04 = Conv2DTranspose(10, kernel_size=(5, 5),dilation_rate=(3,3))(transpose_04)       
    transpose_04 = Activation(activation_hidden)(transpose_04)
    transpose_04 = Conv2DTranspose(10, kernel_size=(3, 3),dilation_rate=(2,2))(transpose_04)       
    transpose_04 = Activation(activation_hidden)(transpose_04)
    transpose_04 = Conv2DTranspose(1, kernel_size=(3, 3),dilation_rate=(2,2))(transpose_04)       
    transpose_04 = Activation(activation_hidden)(transpose_04)
    
    concatenate_05 = concatenate([concatenate_01, transpose_04], axis=3)
    concatenate_05 = Conv2D(1, kernel_size=(1, 1),padding="same")(concatenate_05)
    output = concatenate_05
    
    rate = output[:,:,:,:1]

    rate = Flatten()(rate)

    
    rate   = Dense(8*8)(rate)
    #max_count      = Dropout(0.1)(max_count)
    

    rate   = Dense(256*256,activation="sigmoid")(rate)
    rate   = tf.keras.layers.Reshape((256,256,1))(rate)

    #rate = Activation(activation_hidden)(rate)
    #input_dist= tf.concat([max_count,prob],axis=-1)
    
    #print(max_count)
    output_dist = tfp.layers.DistributionLambda(name="DistributionLayer",
        make_distribution_fn=lambda t: tfd.Independent(tfp.distributions.Poisson(
            rate=tf.math.softplus(t[...])),
                                                      reinterpreted_batch_ndims=-1 ))
    output = output_dist(rate)
    #output = output_dist(prob)
    model = Model(inputs=inputs, outputs=output)

    
    return model


In [4]:
optimizer = Adam( lr = 1e-4 )
trainer = Trainer(negative_Binomial_Unet_BigBrainTime,
                    lossfunction=NLL,
                    pathToData=(train,test),
                    batch_size = batch_size,
                    optimizer=optimizer,
                    dimension = dimension,
                    channels = channels,
                    metrics = ["mse","mae"])
layer_name="DistributionLayer"
print(trainer.model.get_layer(layer_name).output)

[Loaded file]  ./model_data/negative_Binomial_Unet_BigBrainTime_function/negative_Binomial_Unet_BigBrainTime_function512x512x5.h5
15
[Loaded file]  ./model_data/negative_Binomial_Unet_BigBrainTime_function/negative_Binomial_Unet_BigBrainTime_function512x512x5history.json
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 5) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 5)  630         input_1[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, 512, 512, 5)  0           conv2d[0][0]                     
____________________

In [ ]:
trainer.fit(15)
#m = trainer.model
#print(m.layers[1].get_weights())

/home/simon/Dokumente/Teamprojekt/DeepRain2/Networks/Utils/transform.py:120: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return img[self.slices]


Epoch 16/30
2479/2479 [==============================] - ETA: 0s - loss: 0.2672 - mse: 195.4861 - mae: 1.1535
Epoch 00016: val_loss improved from inf to 0.36341, saving model to ./model_data/negative_Binomial_Unet_BigBrainTime_function/model-016-0.267209-0.363407.h5
2479/2479 [==============================] - 1430s 577ms/step - loss: 0.2672 - mse: 195.4861 - mae: 1.1535 - val_loss: 0.3634 - val_mse: 226.1078 - val_mae: 1.4646
Epoch 17/30
2479/2479 [==============================] - ETA: 0s - loss: 0.2646 - mse: 156.6113 - mae: 1.1072
Epoch 00017: val_loss improved from 0.36341 to 0.36330, saving model to ./model_data/negative_Binomial_Unet_BigBrainTime_function/model-017-0.264642-0.363299.h5
2479/2479 [==============================] - 1422s 574ms/step - loss: 0.2646 - mse: 156.6113 - mae: 1.1072 - val_loss: 0.3633 - val_mse: 89.5594 - val_mae: 1.0843
Epoch 18/30
2479/2479 [==============================] - ETA: 0s - loss: 0.2642 - mse: 157.7127 - mae: 1.1024
Epoch 00018: val_loss imp

In [ ]:
m = trainer.model
w = m.layers[1].get_weights()
print(w)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.pyplot import figure
import numpy as np
%matplotlib inline
history = trainer.history
def plotHistory():
    # Plot training & validation accuracy values
    plt.plot(history['mse'])
    plt.plot(history['val_mse'])
    plt.title('Model MSE')
    plt.ylabel('MSE')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
plotHistory()

In [ ]:
model = trainer.model
pred = None
label = None
pred_ = 20
test.on_epoch_end()
mean = []
stdev = []
label = []
prediction = []
for x,y in test:
    for i in range(batch_size):
        if y[i,:,:,:].max() >= 0:
            pred = model(np.array([x[i,:,:,:]]))
            prediction.append(pred)
            mean.append(pred.mean())
            stdev.append(pred.stddev())
            label.append(y[i,:,:,:])
    if len(label) >= pred_:
        break
        


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.pyplot import figure
import numpy as np
%matplotlib inline
fig, axes = plt.subplots(20, 3, figsize=(16,8*10),dpi=64)
#fig.set_title(["mean","stdev","label"])
for batch,img in enumerate(mean):
    if batch == 20:
        break
    axes[batch,0].imshow(stdev[batch][0,:,:,0],cmap="gray")
    axes[batch,1].imshow(img[0,:,:,0],cmap="gray")
    axes[batch,2].imshow(label[batch][:,:,0],cmap="gray")

In [ ]:
def map_Dist_To_Rain(prediction,label):
    
    x,y,t      = prediction[0][0,:,:,:].shape
    rain_true  = np.zeros((x,y,t))
    rain_false = rain_true.copy()
    
    
    
    ones = np.ones((x,y))
    zeros = np.zeros((x,y,t))
    value_map = np.array([ones * i for i in range(256)])
    value_map = value_map.transpose(1,2,0)
    for i,pred in enumerate(prediction):
        ones = np.ones((x,y,t)) * 3/255
        #print((np.array(pred.prob(ones))).max(axis=-1) )
        probs = np.array(pred.prob(value_map))
        print(label[i].max())
        max_probs = probs.argmax(axis=-1)
        #print(probs[0,1,1,:].argmax())

        #print(probs.argmax(axis=-1))
        #print(probs.max(axis=-1))
        for j in range(256):
            for k in range(256):
                at = np.argmax(probs[0,k,j,:])
                if at > 0:
                #at = probs[0,k,j,:] .sum()
                    print("AATT",at)
        #    print(j,"min:",probs[:,:,:,j].min(),"max:",probs[:,:,:,j].max())
        #print(probs.shape)
        #print(probs)
        #print(pred.prob(0.0))
        print(np.array(pred.prob(label[i].max())).max() )
        print(np.array(pred.prob(label[i].max())).min() )
        if label[i].max() == 0:
            continue

        print(pred)
        s = (1-pred.cdf(zeros))
        
        #print(np.array(s).max(),np.array(probs.mean()).max(),label[i].max())
        
        plt.imshow(max_probs[0,:,:],cmap="gray")
        plt.show()
        plt.imshow(s[0,:,:,0],cmap="gray")
        plt.show()
        plt.imshow(label[i][:,:,0],cmap="gray")
        plt.show()
        break
    

map_Dist_To_Rain(prediction,label)
#print(prediction[0])

In [ ]:
a = np.arange(255)
print(label[0].max())
idx = np.where(label[0] == label[0].max() )
print(idx)
pred = prediction[0]
for i in range(255):
    a = np.ones((256,256,1)) * i
    #print(i,np.array(pred.prob(a)).max() )
#print(np.array(pred.mean())[33,28])
y = []
x = []
for i in range(256):
    x.append(i)
    #print(i,np.array(pred.prob(i))[0,33,28,:] )
    y.append(np.array(pred.prob(i))[0,idx[0],idx[1],0][0])
print(np.array(pred.mean())[0,idx[0],idx[1],0][0])

import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

sns.set(color_codes=True)
sns.set(rc={'figure.figsize':(200,8.27)})
sns.barplot(x,y)



In [ ]:

propability = np.array(prediction[0].prob(label[0]))
mean = np.array(prediction[0].mean())


for i in range(256):
    for j in range(256):
        print(propability[0,i,j,0],"\t",mean[0,i,j,0],"\t",label[0][i,j,0],)


In [ ]:
print(propability.mean())